In [3]:
a = """'isSale : "True",',
'listingID : "4750352",',
'mapLat: "3.165975",',
'mapLon: "101.642193",',
'buildingID: "2513",',
'buildingName : "Kiaramas Danai",',
'propertyType: "Condominium",',
'state: "Kuala Lumpur",',
'area: "Mont Kiara",',
'township: "jalan desa kiara",',
'address: "Jalan Desa Kiara, Mont Kiara, Kuala Lumpur",',
'ownerName : "Karen Leong",',
'ownerPhone: "012-278 9972",',
'userName: "",',
'userEmail: "",',
'userCellPhone: "",',
'price: "RM 2,150,000",',
'displayTitle: "Kiaramas Danai, Mont Kiara",',
'image: "http://pictures.my.ippstatic.com/realtors/images/240/25980/96b64ba308b945c7b666094508950ae6.jpg",'
'askingPrice: "2150000.0000",',
'protype: "CO",',
'stateCode: "KL",',
'cauroselAgentListing: "True",',
'cauroselAgentArea: "False",',
'reportListingURL: "/iframe/listing_not_available.aspx?pid=4750352&ag=Karen Leong (25980&pd=05//08//2016"']"""

In [11]:
[i.split(': ')[0].strip().replace('/','').replace("'",'') for i in a.splitlines()]

['isSale',
 'listingID',
 'mapLat',
 'mapLon',
 'buildingID',
 'buildingName',
 'propertyType',
 'state',
 'area',
 'township',
 'address',
 'ownerName',
 'ownerPhone',
 'userName',
 'userEmail',
 'userCellPhone',
 'price',
 'displayTitle',
 'image',
 'askingPrice',
 'protype',
 'stateCode',
 'cauroselAgentListing',
 'cauroselAgentArea',
 'reportListingURL']

In [12]:
a = """KL
JO
KE
KT
MA
NS
OT
PA
PE
PK
PL
PJ
SA
SW
SE
TR"""

In [13]:
a.splitlines()

['KL',
 'JO',
 'KE',
 'KT',
 'MA',
 'NS',
 'OT',
 'PA',
 'PE',
 'PK',
 'PL',
 'PJ',
 'SA',
 'SW',
 'SE',
 'TR']

In [10]:
import pandas as pd
from collections import Counter

In [422]:
df = pd.read_csv('data/iprop2_backup.csv')
df = df[['PostedDate','listingID', 'buildingID', 'stateCode', 'area', 'township', 'displayTitle', 'address', 
    'propertyType','BuiltUp', 'mapLon','mapLat', 
    'askingPrice', 'AskingPricepsf', 'Bedrooms', 'Bathrooms', 'CarParks', 'Occupancy', 'FacingDirection', 
    'Tenure', 'Facilities','Furnishing']]

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (4,9,14,20,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [423]:
def clean_builtup(x):
    try:
        return float(x.replace(' sq. ft.','').replace(',','').replace(',',''))
    except:
        return x.replace(' sq. ft.','').replace(',','').replace(',','')
def convert_to_sqft(x):
    if type(x)==str:
        if 'sq. m' in x:
            return float(x.replace(' sq. m', '')) * 10.7639
        elif 'acres' in x:
            return float(x.replace(' acres', '')) * 43560
    else:
        return x
df['BuiltUp'] = df['BuiltUp'].map(lambda x: clean_builtup(x) if type(x)==str else x).map(convert_to_sqft).map(float)
df['PostedDate'] = pd.to_datetime(df['PostedDate'])

In [424]:
def cleanBedrooms(x):
    if type(x) == str:
        if '+' in x:
            return float(x.split('+')[0]) + float(x.split('+')[0])/3
        elif x == 'Studio':
            return 1
        elif x == '20 Above':
            return 20
        else:
            return int(x)
    else:
        return x
df['Bedrooms'] = df['Bedrooms'].map(cleanBedrooms).map(float)

In [425]:
statemapping = {'JO': 'Johor',
 'KE': 'Kedah',
 'KL': 'Kuala Lumpur',
 'KT': 'Kelantan',
 'MA': 'Melaka',
 'NS': 'Negeri Sembilan',
 'OT': 'Other',
 'PA': 'Pahang',
 'PE': 'Penang',
 'PJ': 'Putrajaya',
 'PK': 'Perak',
 'PL': 'Perlis',
 'SA': 'Sabah',
 'SE': 'Selangor',
 'SW': 'Sarawak',
 'TR': 'Terengganu'}
df['state'] = df['stateCode'].map(statemapping)

df['price_per_sqft'] = df['askingPrice'] / df['BuiltUp']
df.drop(['stateCode','AskingPricepsf'],axis=1,inplace=True)

In [426]:
import re

In [427]:
def standardize_fac(x):
    if '24' in x and 'security' in x:
        return '24hr security'
    elif 'cover' in x and 'parking' in x:
        return 'covered parking'
    elif 'play' in x and 'ground' in x:
        return 'playground'
    elif 'swimming' in x or 'pool' in x:
        return 'swimming pool'
    elif 'gym' in x:
        return 'gymnasium'
    elif 'barbecue' in x or 'bbq' in x:
        return 'barbecue area'
    elif 'jogging' in x:
        return 'jogging track'
    elif 'mini' in x and 'market' in x:
        return 'mini market'
    elif 'wading' in x and 'pool' in x:
        return 'wading pool'
    elif 'cafe' in x:
        return 'cafeteria'
    elif 'sauna' in x:
        return 'sauna'
    elif 'club' in x and 'house' in x:
        return 'club house'
    elif 'business' in x:
        return 'business centre'
    elif 'jacuzzi' in x:
        return 'jacuzzi'
    elif 'tennis' in x:
        return 'tennis court'
    elif 'squash' in x:
        return 'squash court',
    elif 'nursery' in x:
        return 'nursery'
    elif 'salon' in x:
        return 'salon'
    elif 'balcony' in x:
        return 'balcony'
    elif ('multi' in x and 'purpose' in x) or ('function' in x and 'room' in x):
        return 'multi-purpose hall'
    elif 'restaurant' in x:
        return 'restaurant'
    elif 'basketball' in x:
        return 'basketball court'
    elif 'badminton' in x:
        return 'badminton court'
    elif 'games' in x:
        return 'games room'
    elif 'landscape' in x:
        return 'landscaped garden'
    elif 'reading' in x:
        return 'reading room'
    elif 'laund' in x:
        return 'laundrette'
    elif 'reflexol' in x:
        return 'reflexology path'
    else:
        return x
top_fac = ['24hr security',
 'covered parking',
 'playground',
 'swimming pool',
 'gymnasium',
 'barbecue area',
 'jogging track',
 'mini market',
 'wading pool',
 'cafeteria',
 'sauna',
 'club house',
 'business centre',
 'jacuzzi',
 'tennis court',
 'squash court',
 'nursery',
 'salon',
 'balcony',
 'multi-purpose hall',
 'restaurant',
 'basketball court',
 'badminton court',
 'games room',
 'landscaped garden',
 'reading room',
 'laundrette',
 'reflexology path']
df['Facilities'] = df['Facilities'].map(lambda x: [standardize_fac(y.lower()) for y in x.lower().split(',')] if type(x)==str else x)

In [428]:
len(top_fac)

28

In [429]:
def check_fac(x):
    if type(x) == list:
        if f in x:
            return 1
        else:
            return 0
    else:
        return 0
    
for f in top_fac:
    df['facility_'+f] = df['Facilities'].map(check_fac)

In [430]:
area_counter = Counter(df['area'])
major_area = [k for k,v in area_counter.items() if v >= 50]
df['area_clean'] = df.apply(lambda x: x['area'] if x['area'] in major_area else x['state'], axis=1)

In [ ]:
cols = """PostedDate	listingID	buildingID	area	township	displayTitle	address	propertyType	BuiltUp	mapLon	mapLat	askingPrice	Bedrooms	Bathrooms	CarParks	Occupancy	FacingDirection	Tenure	Facilities	Furnishing	state	price_per_sqft	facility_24hr security	facility_covered parking	facility_playground	facility_swimming pool	facility_gymnasium	facility_barbecue area	facility_jogging track	facility_mini market	facility_wading pool	facility_cafeteria	facility_sauna	facility_club house	facility_business centre	facility_jacuzzi	facility_tennis court	facility_squash court	facility_nursery	facility_salon	facility_balcony	facility_multi-purpose hall	facility_restaurant	facility_basketball court	facility_badminton court	facility_games room	facility_landscaped garden	facility_reading room	facility_laundrette	facility_reflexology path"""

In [456]:
df[cols.split('	')].to_excel('iproperty_full.xlsx',index=False)

In [162]:
df[df['area_clean']=='Seberang Jaya'][['PostedDate','listingID', 'buildingID', 'state', 'area', 'township', 'displayTitle', 'address', 
    'propertyType','BuiltUp', 'askingPrice', 'price_per_sqft']].sort_values('price_per_sqft',ascending=False)

,PostedDate,listingID,buildingID,state,area,township,displayTitle,address,propertyType,BuiltUp,askingPrice,price_per_sqft
17686,2016-07-28,4774814,NaN,Penang,Seberang Jaya,Seberang Jaya,"Taman Seri Arowana, Seberang Jaya",Off Jalan Seberang Jaya,2-sty Terrace/Link House,800.0,600000.0,750.000000
31270,2016-07-18,4784183,NaN,Penang,Seberang Jaya,NaN,Seberang Jaya,NaN,2-sty Terrace/Link House,800.0,500000.0,625.000000
36940,2016-08-07,4736193,NaN,Penang,Seberang Jaya,NaN,Seberang Jaya,NaN,Semi-detached House,2100.0,1100000.0,523.809524
14579,2016-01-08,4678598,NaN,Penang,Seberang Jaya,NaN,Seberang Jaya,NaN,2-sty Terrace/Link House,1600.0,790000.0,493.750000
14618,2016-01-08,2863484,NaN,Penang,Seberang Jaya,"Villa Primero,Taman Naluri Indah",Seberang Jaya,NaN,2-sty Terrace/Link House,1800.0,780000.0,433.333333
31173,2016-07-18,4586891,NaN,Penang,Seberang Jaya,NaN,"The Signature Condo, Seberang Jaya",Jalan Baru,Condominium,1100.0,470000.0,427.272727
25119,2016-07-22,4577976,NaN,Penang,Seberang Jaya,NaN,"The Signature Condo, Seberang Jaya",Jalan Baru,Condominium,1100.0,470000.0,427.272727
262,2016-02-08,4831392,NaN,Penang,Seberang Jaya,NaN,Seberang Jaya,NaN,Semi-detached House,2350.0,1000000.0,425.531915
4351,2016-05-08,4648013,NaN,Penang,Seberang Jaya,Seberang Jaya,"Taman Tun Hussein Onn, Seberang Jaya",Jalan jenahak,Semi-detached House,2350.0,1000000.0,425.531915
23635,2016-07-25,4639499,NaN,Penang,Seberang Jaya,Seberang Jaya,"Taman Tun Hussein Oon, Seberang Jaya",Jalan Jenahak,Semi-detached House,2350.0,1000000.0,425.531915


72

In [18]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso

In [166]:
en = ElasticNet()

In [442]:
lasso= Lasso(alpha=0.11)

In [443]:
my_x = df[(~pd.isnull(df['price_per_sqft'])) & (df['price_per_sqft'].between(100,5000)) ][
    ['area_clean','propertyType','CarParks','Occupancy','FacingDirection','Tenure','Furnishing']+
   [c for c in df.columns if 'facility_' in c]]

In [444]:
my_x = pd.get_dummies(my_x)
my_x = my_x.fillna(my_x.mean())

In [445]:
my_y = df[ (~pd.isnull(df['price_per_sqft'])) & (df['price_per_sqft'].between(100,5000)) ]['price_per_sqft']

In [446]:
lasso.fit(my_x, my_y )

Lasso(alpha=0.11, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [458]:
lasso.intercept_

451.48813090335716

In [447]:
coef_dict = {col:coef for coef, col in zip(lasso.coef_, my_x.columns)}

In [448]:
feature_coef = [(i,coef_dict[i]) for i in sorted(coef_dict, key=lambda x:coef_dict[x], reverse=True) ]

In [449]:
feature_df = pd.DataFrame(feature_coef, columns=['Feature','Coefficient (Price per SqFt)'])

In [450]:
feature_df['Feature_Category'] = feature_df['Feature'].map(lambda x: x.rsplit('_',maxsplit=1)[0])
feature_df['Feature_Category'] = feature_df['Feature_Category'].map(lambda x: 'Area' if x=='area_clean' else x)
feature_df['Feature_Value'] = feature_df['Feature'].map(lambda x: 'FOR EACH' if len(x.rsplit('_',maxsplit=1))==1 else x.rsplit('_',maxsplit=1)[1] ) 

In [451]:
feature_df = feature_df.sort_values(['Feature_Category','Coefficient (Price per SqFt)', 'Feature_Value'], ascending=False).drop('Feature', axis=1)

In [452]:
feature_df.to_csv('property_feature_coefficient.csv', index=False)

In [58]:
df.dtypes

PostedDate                     datetime64[ns]
listingID                               int64
buildingID                            float64
area                                   object
township                               object
displayTitle                           object
address                                object
propertyType                           object
BuiltUp                               float64
mapLon                                float64
mapLat                                float64
askingPrice                           float64
Bedrooms                              float64
Bathrooms                             float64
CarParks                              float64
Occupancy                              object
FacingDirection                        object
Tenure                                 object
Facilities                             object
Furnishing                             object
state                                  object
price_per_sqft                    

In [11]:
fac_list = list(df['Facilities'].unique())

In [14]:
fac_list[:1]

['Covered Parking,Playground,Swimming Pool,24hr Security,Cafeteria,Gymnasium,Squash Court,Wading Pool']

In [28]:
from collections import Counter

In [30]:
fac_counter = Counter(f.lower() for ftype in fac_list if type(ftype)==str for f in ftype.split(',') )

In [41]:
fac_counter['24-hour security']

47

In [35]:
sorted(fac_counter, key=lambda x: fac_counter[x], reverse=True)

['24hr security',
 'covered parking',
 'playground',
 'swimming pool',
 'gymnasium',
 'barbecue area',
 'jogging track',
 'mini market',
 'wading pool',
 'cafeteria',
 'sauna',
 'club house',
 'business centre',
 'jacuzzi',
 'tennis court',
 'squash court',
 'nursery',
 'salon',
 'balcony',
 'multi-purpose hall',
 'restaurant',
 'open car park',
 'function room',
 'basketball court',
 'badminton court',
 'games room',
 'landscaped garden',
 'multipurpose hall',
 'reading room',
 'lift',
 '24-hour security',
 'game room',
 'surau (male & female)',
 'reflexology path',
 'fun pool',
 'surau',
 'meeting room',
 'perimeter fencing',
 'pool deck',
 'library',
 'sky lounge',
 'basement car park',
 'table tennis',
 'launderette',
 'lounge',
 'community hall',
 'badminton hall',
 'recreation room',
 'sky garden',
 'children?s playground',
 'sun deck',
 'laundrette',
 'school',
 'putting green',
 'hospital',
 'bank',
 'clubhouse',
 'picnic area',
 'reading lounge',
 '-',
 'fitness corner',
 'mul